In [1]:
# Import the search space
import sys
sys.path.append('../')

from naslib.search_spaces import NasBench201SearchSpace

In [2]:
# Create a new search space object. This object doesn't have an architecture
# assigned to it yet
graph = NasBench201SearchSpace(n_classes=10)

In [3]:
# Sample a random architecture
# You can call this method only once.
graph.sample_random_architecture()

# Get the NASLib representation of this architecture
graph.get_hash()

(1, 3, 0, 3, 1, 4)

In [4]:
# This graph is now a NAS-Bench-201 model, which can be used for training
# Forward pass some dummy data through it to see it in action

import torch

x = torch.randn(5, 3, 32, 32) # (Batch_size, Num_channels, Height, Width)
logits = graph(x)

print(logits.shape)

torch.Size([5, 10])


In [5]:
# Import code to convert NASLib graph to the original NAS-Bench-201 representation
from naslib.search_spaces.nasbench201.conversions import convert_naslib_to_str as convert_naslib_nb201_to_str

# Get the string representation of this model
convert_naslib_nb201_to_str(graph)

'|none~0|+|nor_conv_1x1~0|nor_conv_1x1~1|+|skip_connect~0|none~1|avg_pool_3x3~2|'

In [6]:
# Mutating an architecture
# First, create a new child_graph
child_graph = NasBench201SearchSpace(n_classes=10)

# Call mutate on the child graph by passing the parent graph to it
child_graph.mutate(parent=graph)

# See the parent and child graph representations
print(f'Parent graph: {graph.get_hash()}')
print(f'Child graph : {child_graph.get_hash()}')

Parent graph: (1, 3, 0, 3, 1, 4)
Child graph : (1, 3, 2, 3, 1, 4)


In [7]:
# Now, let's load the queryable tabular NAS-Bench-201 API
# This API has the training metrics of all the 15625 models in the search space
# such as train and validation accuracies/losses at every epoch

from naslib.utils import get_dataset_api
benchmark_api = get_dataset_api(search_space='nasbench201', dataset='cifar10')

In [8]:
# With the NAS-Bench-201 API, we can now query, say, the validation performance of any NB201 model
# Without it, we would have to train the model from scratch to get this information

# First, import the Metric enum
from naslib.search_spaces.core import Metric

# Metric has, among others, these values:
# Metric.TRAIN_ACCURACY
# Metric.VAL_ACCURACY
# Metric.TRAIN_LOSS
# Metric.TEST_LOSS
# Metric.TRAIN_TIME

train_acc_parent = graph.query(metric=Metric.TRAIN_ACCURACY, dataset='cifar10', dataset_api=benchmark_api)
val_acc_parent = graph.query(metric=Metric.VAL_ACCURACY, dataset='cifar10', dataset_api=benchmark_api)

print('Performance of parent model')
print(f'Train accuracy: {train_acc_parent}')
print(f'Validation accuracy: {val_acc_parent}')

# TODO: Query the train and validation performance of the child model
# train_acc_parent = ...
# val_acc_parent = ...

# print('Performance of child model')
# print(f'Train accuracy: {train_acc_child}')
# print(f'Validation accuracy: {val_acc_child}')

Performance of parent model
Train accuracy: 98.98800000976563
Validation accuracy: 86.11


In [9]:
# TODO: 
# 1. Sample a random NAS-Bench-301 model
# 2. Get the NASLib and genotype representations of the model
# 3. Query the predicted performance of the model (loading the NB301 benchmark API might take some time)
# 4. Mutate the model
# 5. Get the NASLib and genotype representations of the model
# 6. Query the predicted performance of the child

from naslib.search_spaces import NasBench301SearchSpace
from naslib.search_spaces.nasbench301.conversions import convert_naslib_to_genotype as convert_naslib_nb301_to_genotype

## Optimizers

In [11]:
import json
import logging
import os

# import the Trainer used to run the optimizer on a given search space
from naslib.defaults.trainer import Trainer
# import the optimizers
from naslib.optimizers import (
    RandomSearch,
    RegularizedEvolution
)
# import the search spaces
from naslib.search_spaces import (
    NasBench101SearchSpace,
    NasBench201SearchSpace,
    NasBench301SearchSpace,
)

from naslib.search_spaces.core.query_metrics import Metric
from naslib import utils
from naslib.utils import get_dataset_api
from naslib.utils.log import setup_logger

from fvcore.common.config import CfgNode # Required to read the config
###### End of imports ######

# The configuration used by the Trainer and Optimizer
config = {
    'dataset': 'cifar10',
    'search': {
        'seed': 0, # 
        'epochs': 5, # Number of epochs (steps) of the optimizer to run
        'fidelity': -1, # 
        'checkpoint_freq': 100,
    },
    'save': 'runs' # folder to save the results to 
}

config = CfgNode.load_cfg(json.dumps(config))

# Make the directories required for search and evaluation
os.makedirs(config['save'] + '/search', exist_ok=True)
os.makedirs(config['save'] + '/eval', exist_ok=True)

# Set up the loggers
logger = setup_logger()
logger.setLevel(logging.INFO)

# See the config
logger.info(f'Configuration is \n{config}')
# logger.info(config)

# Set up the seeds
utils.set_seed(9002)

# Instantiate the search space and get its benchmark API
search_space = NasBench201SearchSpace()
dataset_api = get_dataset_api('nasbench201', 'cifar10')

# Instantitate the optimizer and adapt the search space to it
optimizer = RandomSearch(config)
optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

# Create a Trainer
trainer = Trainer(optimizer, config)

# Perform the search
trainer.search(resume_from="", report_incumbent=False)

# Get the results of the search
search_trajectory = trainer.search_trajectory
print('Train accuracies:', search_trajectory.train_acc)
print('Validation accuracies:', search_trajectory.valid_acc)

# Get the validation performance of the best model found in the search phase
best_model_val_acc = trainer.evaluate(dataset_api=dataset_api, metric=Metric.VAL_ACCURACY)
best_model_val_acc

[09/13 10:53:16 naslib]: Configuration is 
dataset: cifar10
save: runs
search:
  checkpoint_freq: 100
  epochs: 5
  fidelity: -1
  seed: 0
[09/13 10:53:17 nl.defaults.trainer]: Beginning search
[09/13 10:53:17 nl.defaults.trainer]: Epoch 0 done. Train accuracy: 99.11600, Validation accuracy: 86.81000
[09/13 10:53:18 nl.defaults.trainer]: Epoch 1 done. Train accuracy: 99.55200, Validation accuracy: 84.74000
[09/13 10:53:18 nl.defaults.trainer]: Epoch 2 done. Train accuracy: 90.96800, Validation accuracy: 84.01000
[09/13 10:53:18 nl.defaults.trainer]: Epoch 3 done. Train accuracy: 99.16800, Validation accuracy: 84.36000
[09/13 10:53:18 nl.defaults.trainer]: Epoch 4 done. Train accuracy: 99.96400, Validation accuracy: 90.21000
[09/13 10:53:18 nl.defaults.trainer]: Saving architectural weight tensors: runs/arch_weights.pt
[09/13 10:53:18 nl.defaults.trainer]: Training finished
Train accuracies: [99.116, 99.5520000024414, 90.96799999023438, 99.16800000732422, 99.964]
Validation accuracies: 

90.21

In [12]:
def update_config(config, optimizer_type, search_space_type, dataset, seed):
    # Dataset being used
    config.dataset = dataset
    
    # Directory to which the results/logs will be saved
    config.save = f"runs/{optimizer_type.__name__}/{search_space_type.__name__}/{dataset}/{seed}"
    
    # Seed used during search phase of the optimizer
    config.search.seed = seed
    
def run_optimizer(optimizer_type, search_space_type, dataset, dataset_api, config, seed):
    # Update the config
    update_config(config, optimizer_type, search_space_type, dataset, seed)

    # Make the results directories
    os.makedirs(config.save + '/search', exist_ok=True)
    os.makedirs(config.save + '/eval', exist_ok=True)

    # Set up the loggers
    logger = setup_logger()
    logger.setLevel(logging.INFO)

     # See the config
    logger.info(f'Configuration is \n{config}')

    # Set up the seed
    utils.set_seed(seed)

    # Instantiate the search space
    n_classes = {
        'cifar10': 10,
        'cifar100': 100,
        'ImageNet16-120': 120
    }
    search_space = search_space_type(n_classes=n_classes[dataset])

    # Get the benchmark API
    logger.info('Loading Benchmark API')
    dataset_api = get_dataset_api(search_space.get_type(), dataset)
    
    # Instantiate the optimizer and adapat the search space to the optimizer
    optimizer = optimizer_type(config)
    optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

    # Create a Trainer
    trainer = Trainer(optimizer, config)

    # Perform the search
    trainer.search(report_incumbent=False)

    # Get the results of the search
    search_trajectory = trainer.search_trajectory
    print('Train accuracies:', search_trajectory.train_acc)
    print('Validation accuracies:', search_trajectory.valid_acc)

    # Get the validation performance of the best model found in the search phase
    best_model_val_acc = trainer.evaluate(dataset_api=dataset_api, metric=Metric.VAL_ACCURACY)
    best_model_val_acc

    best_model = optimizer.get_final_architecture()

    return search_trajectory, best_model, best_model_val_acc

In [13]:
# Set the optimizer and search space types
# They will be instantiated inside run_optimizer
optimizer_type = RegularizedEvolution # {RegularizedEvolution, RandomSearch}
search_space_type = NasBench201SearchSpace # {NasBench101SearchSpace, NasBench201SearchSpace, NasBench301SearchSpace}

# Set the dataset
dataset = 'cifar100' # cifar10 for NB101 and NB301, {cifar100, ImageNet16-120} for NB201

# The configuration used by the Trainer and Optimizer
# The missing information will be populated inside run_optimizer
config = {
    'search': {
        # Required by Trainer
        'epochs': 100,
        'checkpoint_freq': 100,
        
        # Required by Random Search optimizer
        'fidelity': -1,
        
        # Required by RegularizedEvolution
        'sample_size': 10,
        'population_size': 30,
    }
}
config = CfgNode.load_cfg(json.dumps(config))

search_trajectory, best_model, best_model_val_acc = run_optimizer(
                                                        optimizer_type,
                                                        search_space_type,
                                                        dataset,
                                                        dataset_api,
                                                        config,
                                                        9001
                                                    )

[09/13 10:53:34 naslib]: Configuration is 
dataset: cifar100
save: runs/RegularizedEvolution/NasBench201SearchSpace/cifar100/9001
search:
  checkpoint_freq: 100
  epochs: 100
  fidelity: -1
  population_size: 30
  sample_size: 10
  seed: 9001
[09/13 10:53:34 naslib]: Loading Benchmark API
[09/13 10:53:35 nl.defaults.trainer]: Beginning search
[09/13 10:53:35 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[09/13 10:53:35 nl.optimizers.discrete.re.optimizer]: Population size 1
[09/13 10:53:35 nl.defaults.trainer]: Epoch 0 done. Train accuracy: 52.72400, Validation accuracy: 49.34000
[09/13 10:53:35 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[09/13 10:53:35 nl.defaults.trainer]: Epoch 1 done. Train accuracy: 79.93000, Validation accuracy: 67.04000
[09/13 10:53:35 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[09/13 10:53:36 nl.defaults.trainer]: Epoch 2 don

In [ ]:
!pwd